In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import re
import pickle
# %matplotlib inline

In [37]:
with open('../data/datafest2018NewApril6.pickle', 'rb') as handle:
    df = pickle.load(handle)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17635296 entries, 0 to 17635295
Data columns (total 28 columns):
date                          datetime64[ns]
companyId                     object
jobId                         object
country                       object
stateProvince                 object
city                          object
avgOverallRating              float64
numReviews                    float64
industry                      object
normTitle                     object
normTitleCategory             object
descriptionCharacterLength    int64
descriptionWordCount          int64
experienceRequired            float64
estimatedSalary               float64
salaryCurrency                object
jobLanguage                   object
supervisingJob                float64
licenseRequiredJob            float64
educationRequirement          object
jobAgeDays                    int64
clicks                        int64
localClicks                   float64
month                  

In [38]:
## Industry lookup table
indus = pd.read_csv('../data/predicted_industries.csv')
indus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13927 entries, 0 to 13926
Data columns (total 2 columns):
companyId        13927 non-null object
industry_pred    13927 non-null object
dtypes: object(2)
memory usage: 217.7+ KB


In [18]:
df.loc[:, ['companyId', 'industry']]

,companyId,industry
0,company00090,NaN
1,company198537,NaN
2,company16408,NaN
3,company42051,NaN
4,company11874,HEALTH_CARE
5,company26118,NaN
6,company10962,NaN
7,company182439,NaN
8,company100952,NaN
9,company39530,NaN


In [39]:
indus.head()
print('There are {} unique companies and {} unique industries'.format(len(indus.companyId.unique()), len(indus.industry_pred.unique())))


,companyId,industry_pred
0,company00003,RETAIL
1,company00017,INSURANCE
2,company00025,RETAIL
3,company00036,RETAIL
4,company00037,CONSULTING_AND_BUSINESS_SERVICES


There are 13927 unique companies and 22 unique industries


In [42]:
%%time
dff = pd.merge(df.loc[:,['companyId', 'industry']], indus, how='left', on='companyId')
# dff.head(10)

CPU times: user 12.3 s, sys: 5.58 s, total: 17.9 s
Wall time: 18.4 s


In [43]:
mask = dff.industry.isnull()
dff.industry[mask] = dff.industry_pred[mask]
mask2 = dff.industry.isnull()
dff.industry[mask2] = 'UNCLASSIFIED'

In [44]:
dff.industry.value_counts(dropna=False)

UNCLASSIFIED                                                                                   12711318
HEALTH_CARE                                                                                     1216778
RETAIL                                                                                           967329
CONSULTING_AND_BUSINESS_SERVICES                                                                 587972
RESTAURANTS_TRAVEL_AND_LEISURE                                                                   491095
EDUCATION_AND_SCHOOLS                                                                            293077
BANKS_AND_FINANCIAL_SERVICES                                                                     245072
HUMAN_RESOURCES_AND_STAFFING                                                                     241104
INTERNET_AND_SOFTWARE                                                                            121440
AUTO                                                            

In [45]:
sum(dff.industry == 'UNCLASSIFIED')/len(dff)

0.7207884687617379

In [47]:
dff.head(50)

,companyId,industry,industry_pred
0,company00000,UNCLASSIFIED,NaN
1,company00001,HEALTH_CARE,NaN
2,company00002,UNCLASSIFIED,NaN
3,company00003,RETAIL,RETAIL
4,company00004,UNCLASSIFIED,NaN
5,company00005,UNCLASSIFIED,NaN
6,company00006,UNCLASSIFIED,NaN
7,company00007,UNCLASSIFIED,NaN
8,company00008,UNCLASSIFIED,NaN
9,company00009,UNCLASSIFIED,NaN


In [48]:
df.industry = dff.industry
df.industry.value_counts()

UNCLASSIFIED                                                                                   12711318
HEALTH_CARE                                                                                     1216778
RETAIL                                                                                           967329
CONSULTING_AND_BUSINESS_SERVICES                                                                 587972
RESTAURANTS_TRAVEL_AND_LEISURE                                                                   491095
EDUCATION_AND_SCHOOLS                                                                            293077
BANKS_AND_FINANCIAL_SERVICES                                                                     245072
HUMAN_RESOURCES_AND_STAFFING                                                                     241104
INTERNET_AND_SOFTWARE                                                                            121440
AUTO                                                            

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17635296 entries, 0 to 17635295
Data columns (total 28 columns):
date                          datetime64[ns]
companyId                     object
jobId                         object
country                       object
stateProvince                 object
city                          object
avgOverallRating              float64
numReviews                    float64
industry                      object
normTitle                     object
normTitleCategory             object
descriptionCharacterLength    int64
descriptionWordCount          int64
experienceRequired            float64
estimatedSalary               float64
salaryCurrency                object
jobLanguage                   object
supervisingJob                float64
licenseRequiredJob            float64
educationRequirement          object
jobAgeDays                    int64
clicks                        int64
localClicks                   float64
month                  

In [50]:
%%time 
with open('../data/datafest2018NewApril6-indus.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

CPU times: user 30.9 s, sys: 1min 1s, total: 1min 32s
Wall time: 1min 56s
